<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M1-airbnb-new-room-inference" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost -U -q
!pip install sklearn -U -q

     |████████████████████████████████| 255.9 MB 48 kB/s 


In [ ]:
import pandas as pd
import numpy as np
import itertools
import pickle
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import shap

In [ ]:
X = pd.read_json('X.json')

In [ ]:
model_xgb = pickle.load(open('model_xgb.pkl','rb'))

In [ ]:
model_xgb

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [ ]:
scaler = pickle.load(open('scaler.pkl','rb'))

In [ ]:
X

neighbourhood_cleansed        room_type  instant_bookable  accommodates  \
0                   Nrrebro  Entire home/apt         -0.506681     -0.738065   
1                   Nrrebro  Entire home/apt         -0.506681      1.617060   
2                  Indre By  Entire home/apt         -0.506681      1.617060   
3               Amager Vest  Entire home/apt         -0.506681      0.439498   
4              Brnshj-Husum     Private room         -0.506681     -0.738065   
...                     ...              ...               ...           ...   
3872                  Valby  Entire home/apt         -0.506681      0.439498   
3873                sterbro  Entire home/apt         -0.506681      0.439498   
3874             Bispebjerg  Entire home/apt         -0.506681     -0.738065   
3875             Bispebjerg  Entire home/apt         -0.506681      0.439498   
3876           Brnshj-Husum  Entire home/apt         -0.506681      0.439498   

      bedrooms      beds  minimum_nights_avg_ntm  
0    -0.625227 -0.689055               -0.027534  
1     0.577156  2.382479                0.841476  
2     1.779539  0.846712               -0.027534  
3     0.577156  1.614595               -0.256222  
4    -0.625227  0.078828               -0.027534  
...        ...       ...                     ...  
3872  0.577156  1.614595               -0.484909  
3873  1.779539  0.846712               -0.027534  
3874 -0.625227 -0.689055               -0.416303  
3875  0.577156  0.078828               -0.256222  
3876  1.779539  0.846712                0.887214  

[3877 rows x 7 columns]

In [ ]:
selected_df = pd.read_json('selected_df.json')

In [ ]:
shap_values = pickle.load(open('shap_values.pkl','rb'))
explainer = shap.TreeExplainer(model_xgb)

In [ ]:
selected_df.iloc[0].values

array(['Nrrebro', 'Entire home/apt', False, 2, 1, 1, 3.0, 877],
      dtype=object)

In [ ]:
ohe = pickle.load(open('ohe.pkl','rb'))

In [ ]:
cats = list(itertools.chain(*ohe.categories_))

In [ ]:
new_df = pd.DataFrame({'neighbourhood_cleansed':'Indre By',
              'room_type':'Entire home/apt'}, index=[0])

In [ ]:
new_values_cat = pd.DataFrame(ohe.transform(new_df), columns = cats , index=[0])

In [ ]:
selected_df

neighbourhood_cleansed        room_type  instant_bookable  accommodates  \
0                    Nrrebro  Entire home/apt             False             2   
10                   Nrrebro  Entire home/apt             False             6   
12                  Indre By  Entire home/apt             False             6   
21               Amager Vest  Entire home/apt             False             4   
22              Brnshj-Husum     Private room             False             2   
...                      ...              ...               ...           ...   
12484                  Valby  Entire home/apt             False             4   
12486                sterbro  Entire home/apt             False             4   
12488             Bispebjerg  Entire home/apt             False             2   
12489             Bispebjerg  Entire home/apt             False             4   
12490           Brnshj-Husum  Entire home/apt             False             4   

       bedrooms  beds  minimum_nights_avg_ntm  price  
0             1     1                     3.0    877  
10            2     5                     6.8   2093  
12            3     3                     3.0   1600  
21            2     4                     2.0   2000  
22            1     2                     3.0    500  
...         ...   ...                     ...    ...  
12484         2     4                     1.0    494  
12486         3     3                     3.0   1123  
12488         1     1                     1.3    900  
12489         2     2                     2.0    725  
12490         3     3                     7.0    750  

[3877 rows x 8 columns]

In [ ]:
new_df = pd.DataFrame({'instant_bookable':False, 
                        'accommodates': 2, 
                       'bedrooms':1, 
                       'beds':3, 
                       'minimum_nights_avg_ntm':7}, index=[0])

In [ ]:
new_values_num = pd.DataFrame(scaler.transform(new_df), columns = new_df.columns, index=[0])

In [ ]:
line_to_pred = new_values_num.join(new_values_cat)

In [ ]:
line_to_pred

instant_bookable  accommodates  bedrooms      beds  minimum_nights_avg_ntm  \
0         -0.506681     -0.738065 -0.625227  0.846712                0.887214   

   Amager Vest  Amager st  Bispebjerg  Brnshj-Husum  Frederiksberg  Indre By  \
0          0.0        0.0         0.0           0.0            0.0       1.0   

   Nrrebro  Valby  Vanlse  Vesterbro-Kongens Enghave  sterbro  \
0      0.0    0.0     0.0                        0.0      0.0   

   Entire home/apt  Private room  
0              1.0           0.0

In [ ]:
model_xgb.predict(line_to_pred)

array([1324.2007], dtype=float32)

In [ ]:
shap_value = explainer.shap_values(line_to_pred)
shap.initjs()
shap.force_plot(explainer.expected_value, shap_value, line_to_pred)